In [ ]:
#4) Evaluate the Model (RMSE & Precision/Recall@K)

from collections import defaultdict

predictions_user = user_cf.test(testset)
predictions_item = item_cf.test(testset)

rmse_user = accuracy.rmse(predictions_user)
rmse_item = accuracy.rmse(predictions_item)

print(f"User-based CF RMSE: {rmse_user:.4f}")
print(f"Item-based CF RMSE: {rmse_item:.4f}")


def precision_recall_at_k(predictions, k=5, threshold=3.5):
    """Return precision and recall at k metrics for each user"""
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = []
    recalls = []

    for user_ratings in user_est_true.values():
        # Sort user predictions by estimated rating
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k = user_ratings[:k]

        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in top_k)
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in top_k)

        precision = n_rel_and_rec_k / n_rec_k if n_rec_k else 0
        recall = n_rel_and_rec_k / n_rel if n_rel else 0

        precisions.append(precision)
        recalls.append(recall)

    return sum(precisions) / len(precisions), sum(recalls) / len(recalls)

precision_user, recall_user = precision_recall_at_k(predictions_user, k=5)
print(f"User-based CF Precision@5: {precision_user:.4f}")
print(f"User-based CF Recall@5: {recall_user:.4f}")

precision_item, recall_item = precision_recall_at_k(predictions_item, k=5)
print(f"Item-based CF Precision@5: {precision_item:.4f}")
print(f"Item-based CF Recall@5: {recall_item:.4f}")